In [1]:
!pip install flask
!pip install flask_cors
!pip install pyngrok

In [2]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

In [6]:
!pip install --upgrade torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 70.6 MB/s eta 0:00:00
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.20.1+cu124
    Uninstalling torchvision-0.20.1+cu124:
      Successfully uninstalled torchvision-0.20.1+cu124
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 2.7.18 requires torch<2.6,>=1.10, but you have torch 2.6.0 which is incompatible.


In [2]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

detection_checkpoint = '/content/drive/Shareddrives/FYP 2024-2025/Phase-2/Lexical Bias Detection/News/saved_models/RoBERTa_Weighted'
detection_tokenizer = AutoTokenizer.from_pretrained("/content/drive/Shareddrives/FYP 2024-2025/Phase-2/Lexical Bias Detection/News/saved_models/RoBERTa_Weighted")
detection_model = AutoModelForSequenceClassification.from_pretrained("/content/drive/Shareddrives/FYP 2024-2025/Phase-2/Lexical Bias Detection/News/saved_models/RoBERTa_Weighted")

detection_model.to("cuda") # cuda model
detection_model.eval() # set in eval mode

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
         

In [3]:
from unsloth import FastLanguageModel
from unsloth.chat_templates import get_chat_template
max_seq_length = 2048
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True

# finetuned phi-3.5 for mitigation
mitigation_model, mitigation_tokenizer = FastLanguageModel.from_pretrained(
    model_name = "/content/drive/Shareddrives/FYP 2024-2025/Phase-2/models/phi-3.5-3.8b-inference",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

mitigation_tokenizer = get_chat_template(
    mitigation_tokenizer,
    chat_template = "phi-3", # Supports zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, unsloth
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt", "system": "system"}, # ShareGPT style
)
# set model for inference
FastLanguageModel.for_inference(mitigation_model)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.2.4: Fast Llama patching. Transformers: 4.48.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.26G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/140 [00:00<?, ?B/s]

Unsloth 2025.2.4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=3072, out_features=3072, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3072, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=3072, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.

In [4]:
def run_detection(text):
  global detection_tokenizer, detection_model
  inputs = detection_tokenizer(text, max_length=256, return_tensors="pt", truncation=True, padding="max_length").to("cuda")
  # inputs = detection_tokenizer(text, max_length=256, return_tensors="pt", truncation=True, padding="max_length")
  with torch.no_grad():
    outputs = detection_model(**inputs)
  logits = outputs.logits
  predictions = torch.argmax(logits, dim=-1)
  del inputs, outputs, logits # manual memory freeing
  return predictions.item()

In [5]:
def run_mitigation(text):
  global mitigation_tokenizer, mitigation_model
  messages = [
    {"from": "system", "value": "You are provided with a linguistically biased sentence from a news article. Linguistic bias refers to the use of words, phrases, or structures that carry a subjective tone, either overly positive or overly negative. Such biases can influence the reader's perception, making the text less neutral. Your task is to rephrase the given sentence to remove or neutralize the bias in such a way that most of its original meaning, content and structure is retained."},
    {"from": "human", "value": "The biased sentence is: " + text}
  ]
  inputs = mitigation_tokenizer.apply_chat_template(
      messages,
      tokenize = True,
      add_generation_prompt = True, # Must add for generation
      return_tensors = "pt",
  ).to("cuda")
  outputs = mitigation_model.generate(input_ids = inputs, max_new_tokens = 256, use_cache = True)
  text_output = mitigation_tokenizer.batch_decode(outputs)[0]
  text_output = text_output.split("<|assistant|>")[1].strip()
  text_output = text_output.split("<|end|>")[0].strip()
  del inputs, outputs # manual memory freeing
  return text_output

In [6]:
import getpass
import os
import threading

from flask import Flask,request,jsonify
from flask_cors import CORS
from pyngrok import ngrok, conf

print("Enter your ngrok authtoken: ")
conf.get_default().auth_token = getpass.getpass()

app = Flask(__name__)
CORS(app)
port = "5000"

# Open a ngrok tunnel to the H2e997xR8f1YudfI1JK8oCypDFMW_eWTWs6hpP7VJATv1GuSyTTP server
public_url = ngrok.connect(port).public_url
print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:{port}\"")

# Update any base URLs to use the public ngrok URL
app.config["BASE_URL"] = public_url

# Define Flask routes
@app.route("/")
def index():
    return "Hello World!"

@app.route("/api/detection", methods=['POST'])
def detection():
  try:
      # Get the input data (list of sentences)
      data = request.json
      sentences = data.get("sentences", [])
      if not isinstance(sentences, list) or not all(isinstance(s, str) for s in sentences):
          return jsonify({"error": "Invalid input. Expecting a JSON object with a 'sentences' list."}), 400
      # Run detection for each sentence
      results = [run_detection(sentence) for sentence in sentences]
      # Return the list of predictions
      return jsonify({"predictions": results})
  except Exception as e:
        return jsonify({"error": str(e)}), 500

@app.route("/api/mitigation", methods = ['POST'])
def mitigation():
  try:
      # Get the input data (biased sentence)
      data = request.json
      sentence = data.get("sentence", "")
      if not isinstance(sentence, str):
          return jsonify({"error": "Invalid input. Expecting a JSON object with a 'sentence' string."}), 400
      # Run mitigation for the sentence
      debiased_sentence = run_mitigation(sentence)
      # Return the debiased sentence
      return jsonify({"debiased_sentence": debiased_sentence})
  except Exception as e:
        return jsonify({"error": str(e)}), 500

# Run Flask app in a thread
def run_app():
    app.run(host="0.0.0.0", port=5000, debug=False)  # Disable debug mode

# Start Flask in a separate thread
thread = threading.Thread(target=run_app)
thread.start()

Enter your ngrok authtoken: 
··········
 * ngrok tunnel "https://d8b0-34-143-177-58.ngrok-free.app" -> "http://127.0.0.1:5000"


In [7]:
url = "https://d8b0-34-143-177-58.ngrok-free.app"

In [8]:
# Test Bias Detection Endpoint

import requests

det_url = url+"/api/detection"
data = {
    "sentences": [
        "This is a neutral sentence.",
        "The government is corrupt and biased.",
        "Technology is advancing rapidly.",
        "The media is completely one-sided."
    ]
}

try:
    response = requests.post(det_url, json=data)
    print("Status Code:", response.status_code)
    print("Raw Response:", response.text)  # Print the raw response
    if response.status_code == 200:
        print("Predictions:", response.json()["predictions"])
    else:
        print("Error:", response.json())
except requests.exceptions.RequestException as e:
    print("Request failed:", e)
except ValueError as e:
    print("Response parsing failed:", e)


INFO:werkzeug:127.0.0.1 - - [07/Feb/2025 14:26:13] "POST /api/detection HTTP/1.1" 200 -


Status Code: 200
Raw Response: {"predictions":[0,1,0,1]}

Predictions: [0, 1, 0, 1]


In [9]:
# Test Bias Mitigation Endpoint

import requests

mit_url = url+"/api/mitigation"
data = {
    "sentence": "The government is corrupt and biased."
}

try:
    response = requests.post(mit_url, json=data)
    print("Status Code:", response.status_code)
    print("Raw Response:", response.text)  # Print the raw response
    if response.status_code == 200:
        print("Predictions:", response.json()["debiased_sentence"])
    else:
        print("Error:", response.json())
except requests.exceptions.RequestException as e:
    print("Request failed:", e)
except ValueError as e:
    print("Response parsing failed:", e)


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
INFO:werkzeug:127.0.0.1 - - [07/Feb/2025 14:26:22] "POST /api/mitigation HTTP/1.1" 200 -


Status Code: 200
Raw Response: {"debiased_sentence":"The government is perceived as corrupt and biased."}

Predictions: The government is perceived as corrupt and biased.
